In this notebook we train and compare three different learning to rank models. 

Each of the models are built using the [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html) package, which is optimised for gradient boosting. 

We models we build are: RankNet, LambdaNet and LambdaRank. 

In [1]:
import xgboost as xgb
import time

## Data processing

In order to implement the XGBoost algorithms the data must be preprocessed in a particular way - XGBoost requires an file that indicates which query each entry in the data frame comes from. If your training file is `train.txt`, the same folder should contain a file named `train.txt.group`, holding the group information. `train.txt.group` will contain a list of numbers indicating the number of feature vectors for each query, in the same order as the appear in `train.txt`. For example, if `train.txt.group` contained [7,8,5] we would know that there are 20 feature vectors in the training set and that the first 7 correspond to one query, the next 8 correspond to a different query and the final five are from a third query. 

There's a bash script in the repo which should be run to compute the group information file for your `train.txt` file. 

We load in the data as a DMatrix, an xgboost data store which is optimised for efficiency and speed when implementing xgboost models. 

In [2]:
training_data = xgb.DMatrix('data/train_dat.txt')
testing_data = xgb.DMatrix('data/test_dat.txt')

[15:47:42] 723412x137 matrix with 98384032 entries loaded from data/train_dat.txt
[15:47:42] 6000 groups are loaded from data/train_dat.txt.group
[15:47:43] 241521x137 matrix with 32846856 entries loaded from data/test_dat.txt
[15:47:43] 2000 groups are loaded from data/test_dat.txt.group


When XGBoost loads in the data, it also checks the folder containing the data for a file of the same name with file extension .txt.group. The .txt.group file contains query id information for the data in the .txt data set. We can see from the logging info printed when we load in the data that the train_dat file contains data from 6000 different queries, and the test_dat.txt contains data from 2000 queries. XGBoost uses this query information both for building models and for evaluating the goodness of the models built.
We can also see from the logging that the data loaded in is of length 137 - that is the 136-long feature vector and the one "score" which is given by the human in the preprocessing stage. In this data set, the score is an integer from 0 to 4, with 0 denoting "this article is not relevant" and 4 denoting "this article is extrememly relevent".

## LambdaRank

In [3]:
params_lr2 = [('objective','rank:pairwise'),('max_depth',2), ('eta',0.1), ('num_boost_round',4), ('seed',404)]

start_lr2 = time.time()           
model_lr2 = xgb.train(params_lr2, training_data)
end_lr2 = time.time()
print(end_lr2-start_lr2)

12.509445190429688


In [4]:
start_lr2p = time.time()
pred_lr2 = model_lr2.predict(testing_data)
end_lr2p = time.time()
print(end_lr2p-start_lr2p)

0.16581416130065918


In [5]:
pred_lr2

array([0.5466446 , 0.4558069 , 0.4558069 , ..., 0.58684486, 0.548023  ,
       0.56519854], dtype=float32)

In [6]:
params_lr6 = [('objective','rank:pairwise'),('max_depth',6), ('eta',0.1), ('num_boost_round',4), ('seed',404)]

start_lr6 = time.time()           
model_lr6 = xgb.train(params_lr6, training_data)
end_lr6 = time.time()
print(end_lr6-start_lr6)

26.857342958450317


In [7]:
start_lr6p = time.time()
pred_lr6 = model_lr6.predict(testing_data)
end_lr6p = time.time()
print(end_lr6p-start_lr6p)

0.1884000301361084


We write the predictions to file. Later we will read them in and compare them across all the models. 

In [8]:
import pandas as pd
pd.DataFrame(pred_lr6).to_csv("data/pred_lr6.txt", header=None, sep=" ")
pd.DataFrame(pred_lr2).to_csv("data/pred_lr2.txt", header=None, sep=" ")